# PaBiRoboy dynamic equations

First, import the necessary functions from SymPy that will allow us to construct time varying vectors in the reference frames.

In [1]:
from __future__ import print_function, division
from sympy import symbols, simplify, Matrix
from sympy import trigsimp
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point, inertia, RigidBody, KanesMethod
from numpy import deg2rad, rad2deg, array, zeros, linspace
from sympy.physics.vector import init_vprinting, vlatex
import numpy as np
from scipy.integrate import odeint
from pydy.codegen.ode_function_generators import generate_ode_function
%matplotlib inline
from matplotlib.pyplot import plot, legend, xlabel, ylabel, rcParams
rcParams['figure.figsize'] = (14.0, 6.0)
import rospy

SymPy has a rich printing system. Here we initialize printing so that all of the mathematical equations are rendered in standard mathematical notation.

In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

## Interial frames

In [3]:
inertial_frame = ReferenceFrame('I')
lower_leg_left_frame = ReferenceFrame('R_1')
upper_leg_left_frame = ReferenceFrame('R_2')
hip_frame = ReferenceFrame('R_3')
upper_leg_right_frame = ReferenceFrame('R_4')
lower_leg_right_frame = ReferenceFrame('R_5')

## Angles

In [4]:
theta0, theta1, theta2, theta3, theta4 = dynamicsymbols('theta0, theta1, theta2, theta3, theta4')

In [5]:
hip_frame.orient(inertial_frame, 'Axis', (theta0, inertial_frame.z))
hip_frame.dcm(inertial_frame)

Matrix([
[ cos(theta0), sin(theta0), 0],
[-sin(theta0), cos(theta0), 0],
[           0,           0, 1]])

In [6]:
upper_leg_left_frame.orient(hip_frame, 'Axis', (theta1, hip_frame.z))
simplify(upper_leg_left_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta1), sin(theta0 + theta1), 0],
[-sin(theta0 + theta1), cos(theta0 + theta1), 0],
[                    0,                    0, 1]])

In [7]:
lower_leg_left_frame.orient(upper_leg_left_frame, 'Axis', (theta2, upper_leg_left_frame.z))
simplify(lower_leg_left_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta1 + theta2), sin(theta0 + theta1 + theta2), 0],
[-sin(theta0 + theta1 + theta2), cos(theta0 + theta1 + theta2), 0],
[                             0,                             0, 1]])

In [8]:
upper_leg_right_frame.orient(hip_frame, 'Axis', (theta3, hip_frame.z))
simplify(upper_leg_right_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta3), sin(theta0 + theta3), 0],
[-sin(theta0 + theta3), cos(theta0 + theta3), 0],
[                    0,                    0, 1]])

In [9]:
lower_leg_right_frame.orient(upper_leg_right_frame, 'Axis', (theta4, upper_leg_right_frame.z))
simplify(lower_leg_right_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta3 + theta4), sin(theta0 + theta3 + theta4), 0],
[-sin(theta0 + theta3 + theta4), cos(theta0 + theta3 + theta4), 0],
[                             0,                             0, 1]])

## Points and Locations

In [10]:
origin = Point('Origin')
ankle_left = Point('AnkleLeft')
knee_left = Point('KneeLeft')
hip_left = Point('HipLeft')
hip_center = Point('HipCenter')
hip_right = Point('HipRight')
knee_right = Point('KneeRight')
ankle_right = Point('AnkleRight')

In [11]:
lower_leg_length, upper_leg_length, hip_length = symbols('l1, l2, l3')

In [12]:
hip_offset_x, hip_offset_y = symbols('hip_x, hip_y')

In [13]:
hip_center.set_pos(origin, (hip_offset_y * inertial_frame.y)+(hip_offset_x * inertial_frame.x))
hip_center.pos_from(origin).express(inertial_frame).simplify()

hip_x*I.x + hip_y*I.y

In [14]:
hip_left.set_pos(hip_center, hip_length/2 * hip_frame.x)
hip_left.pos_from(origin).express(inertial_frame).simplify()

(hip_x + l3*cos(theta0)/2)*I.x + (hip_y + l3*sin(theta0)/2)*I.y

In [15]:
hip_right.set_pos(hip_center, -hip_length/2 * hip_frame.x)
hip_right.pos_from(origin).express(inertial_frame).simplify()

(hip_x - l3*cos(theta0)/2)*I.x + (hip_y - l3*sin(theta0)/2)*I.y

In [16]:
knee_left.set_pos(hip_left, upper_leg_length * -upper_leg_left_frame.y)
knee_left.pos_from(origin).express(inertial_frame).simplify()

(hip_x + l2*sin(theta0 + theta1) + l3*cos(theta0)/2)*I.x + (hip_y - l2*cos(theta0 + theta1) + l3*sin(theta0)/2)*I.y

In [17]:
knee_right.set_pos(hip_right, upper_leg_length * -upper_leg_right_frame.y)
knee_right.pos_from(origin).express(inertial_frame).simplify()

(hip_x + l2*sin(theta0 + theta3) - l3*cos(theta0)/2)*I.x + (hip_y - l2*cos(theta0 + theta3) - l3*sin(theta0)/2)*I.y

In [18]:
ankle_left.set_pos(knee_left, lower_leg_length * -lower_leg_left_frame.y)
ankle_left.pos_from(origin).express(inertial_frame).simplify()

(hip_x + l1*sin(theta0 + theta1 + theta2) + l2*sin(theta0 + theta1) + l3*cos(theta0)/2)*I.x + (hip_y - l1*cos(theta0 + theta1 + theta2) - l2*cos(theta0 + theta1) + l3*sin(theta0)/2)*I.y

In [19]:
ankle_right.set_pos(knee_right, lower_leg_length * -lower_leg_right_frame.y)
ankle_right.pos_from(origin).express(inertial_frame).simplify()

(hip_x + l1*sin(theta0 + theta3 + theta4) + l2*sin(theta0 + theta3) - l3*cos(theta0)/2)*I.x + (hip_y - l1*cos(theta0 + theta3 + theta4) - l2*cos(theta0 + theta3) - l3*sin(theta0)/2)*I.y

In [20]:
lower_leg_left_com_length = lower_leg_length/2
upper_leg_left_com_length = upper_leg_length/2
hip_com_length = hip_length/2
upper_leg_right_com_length = upper_leg_length/2
lower_leg_right_com_length = lower_leg_length/2

lower_leg_left_mass_center = Point('L_COMleft')
upper_leg_left_mass_center = Point('U_COMleft')
hip_mass_center = Point('H_COMleft')
upper_leg_right_mass_center = Point('U_COMright')
lower_leg_right_mass_center = Point('L_COMright')

In [21]:
lower_leg_left_mass_center.set_pos(ankle_left, lower_leg_left_com_length * lower_leg_left_frame.x)
lower_leg_left_mass_center.pos_from(origin).express(inertial_frame).simplify()

(hip_x + l1*sin(theta0 + theta1 + theta2) + l1*cos(theta0 + theta1 + theta2)/2 + l2*sin(theta0 + theta1) + l3*cos(theta0)/2)*I.x + (hip_y + l1*sin(theta0 + theta1 + theta2)/2 - l1*cos(theta0 + theta1 + theta2) - l2*cos(theta0 + theta1) + l3*sin(theta0)/2)*I.y

In [22]:
upper_leg_left_mass_center.set_pos(knee_left, upper_leg_left_com_length * upper_leg_left_frame.x)
upper_leg_left_mass_center.pos_from(origin).express(inertial_frame).simplify()

(hip_x + l2*sin(theta0 + theta1) + l2*cos(theta0 + theta1)/2 + l3*cos(theta0)/2)*I.x + (hip_y + l2*sin(theta0 + theta1)/2 - l2*cos(theta0 + theta1) + l3*sin(theta0)/2)*I.y

In [23]:
hip_mass_center.set_pos(hip_center, 0 * hip_frame.x)
hip_mass_center.pos_from(origin).express(inertial_frame).simplify()

hip_x*I.x + hip_y*I.y

In [24]:
upper_leg_right_mass_center.set_pos(hip_right, upper_leg_right_com_length * upper_leg_right_frame.x)
upper_leg_right_mass_center.pos_from(origin).express(inertial_frame).simplify()

(hip_x + l2*cos(theta0 + theta3)/2 - l3*cos(theta0)/2)*I.x + (hip_y + l2*sin(theta0 + theta3)/2 - l3*sin(theta0)/2)*I.y

In [25]:
lower_leg_right_mass_center.set_pos(knee_right, lower_leg_right_com_length * lower_leg_right_frame.x)
lower_leg_right_mass_center.pos_from(origin).express(inertial_frame).simplify()

(hip_x + l1*cos(theta0 + theta3 + theta4)/2 + l2*sin(theta0 + theta3) - l3*cos(theta0)/2)*I.x + (hip_y + l1*sin(theta0 + theta3 + theta4)/2 - l2*cos(theta0 + theta3) - l3*sin(theta0)/2)*I.y

# Jacobian

In [26]:
F0 = hip_center.pos_from(ankle_left).express(inertial_frame).simplify().to_matrix(inertial_frame)
F0 = Matrix([F0[0], F0[1]])
F0

Matrix([
[-l1*sin(theta0 + theta1 + theta2) - l2*sin(theta0 + theta1) - l3*cos(theta0)/2],
[ l1*cos(theta0 + theta1 + theta2) + l2*cos(theta0 + theta1) - l3*sin(theta0)/2]])

In [27]:
F1 = hip_center.pos_from(ankle_right).express(inertial_frame).simplify().to_matrix(inertial_frame)
F1 = Matrix([F1[0], F1[1]])
F1

Matrix([
[-l1*sin(theta0 + theta3 + theta4) - l2*sin(theta0 + theta3) + l3*cos(theta0)/2],
[ l1*cos(theta0 + theta3 + theta4) + l2*cos(theta0 + theta3) + l3*sin(theta0)/2]])

In [28]:
J_ankleLeft = F0.jacobian([theta1, theta2])
J_ankleLeft

Matrix([
[-l1*cos(theta0 + theta1 + theta2) - l2*cos(theta0 + theta1), -l1*cos(theta0 + theta1 + theta2)],
[-l1*sin(theta0 + theta1 + theta2) - l2*sin(theta0 + theta1), -l1*sin(theta0 + theta1 + theta2)]])

In [29]:
J_ankleRight = F1.jacobian([theta3, theta4])
J_ankleRight

Matrix([
[-l1*cos(theta0 + theta3 + theta4) - l2*cos(theta0 + theta3), -l1*cos(theta0 + theta3 + theta4)],
[-l1*sin(theta0 + theta3 + theta4) - l2*sin(theta0 + theta3), -l1*sin(theta0 + theta3 + theta4)]])

In [30]:
J = J_ankleLeft.row_join(J_ankleRight)
J

Matrix([
[-l1*cos(theta0 + theta1 + theta2) - l2*cos(theta0 + theta1), -l1*cos(theta0 + theta1 + theta2), -l1*cos(theta0 + theta3 + theta4) - l2*cos(theta0 + theta3), -l1*cos(theta0 + theta3 + theta4)],
[-l1*sin(theta0 + theta1 + theta2) - l2*sin(theta0 + theta1), -l1*sin(theta0 + theta1 + theta2), -l1*sin(theta0 + theta3 + theta4) - l2*sin(theta0 + theta3), -l1*sin(theta0 + theta3 + theta4)]])

In [31]:
# constraint = F1 - offset
# J_ankleRight = constraint.jacobian([theta0, theta1, theta2, theta3, theta4])
# J_ankleRight

In [32]:
values = {lower_leg_length: 0.4, upper_leg_length: 0.54, hip_length: 0.2, theta0: 0.0, theta1: 0.1, theta2: 0.1, theta3: 0.1, theta4: 0.1}
J.subs(values).pinv()

Matrix([
[-1.84260030607042,  9.08983268364918],
[ 3.09260030607042, -21.5481382127232],
[-1.84260030607042,  9.08983268364918],
[ 3.09260030607042, -21.5481382127232]])

In [33]:
kp=5

In [34]:
ankle_left_pos_equation = ankle_left.pos_from(origin).express(inertial_frame).simplify()
knee_left_pos_equation = knee_left.pos_from(origin).express(inertial_frame).simplify()
hip_left_pos_equation = hip_left.pos_from(origin).express(inertial_frame).simplify()
hip_right_pos_equation = hip_right.pos_from(origin).express(inertial_frame).simplify()
knee_right_pos_equation = knee_right.pos_from(origin).express(inertial_frame).simplify()
ankle_right_pos_equation = ankle_right.pos_from(origin).express(inertial_frame).simplify()
hip_center_pos_equation = hip_center.pos_from(origin).express(inertial_frame).simplify()

In [35]:
values = {lower_leg_length: 0.48, upper_leg_length: 0.54, hip_length: 0.175, theta0: 1, theta1: 0 , theta2: 0, theta3: 0, theta4: 0}
hip_center_pos_equation.subs(values).to_matrix(inertial_frame)

Matrix([
[hip_x],
[hip_y],
[    0]])

In [ ]:
from roboy_communication_middleware.msg import JointStatus, JointCommand, JointAngle
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point as geometry_msgsPoint
from geometry_msgs.msg import Pose
rospy.init_node('PaBiRoboy_dynamics', anonymous=True)
jointAngleOffset = Matrix([0,0,0,0])

class PaBiRoboy_DanceController:
    jointCommand_pub = rospy.Publisher('/roboy/middleware/JointCommand', JointCommand, queue_size=1)
    visualization_pub = rospy.Publisher('/visualization_marker', Marker, queue_size=1)
    msg_viz = Marker()
    hipCenter = Pose()
    hip_desired = Matrix([0.6, 0.8])
    def __init__(self):
        self.msg_viz.header.frame_id = "world"
        self.msg_viz.type = Marker.ARROW
        self.msg_viz.scale.x = 0.005
        self.msg_viz.scale.y = 0.05
        self.msg_viz.scale.z = 0.05
        self.msg_viz.color.a = 0.5
        self.msg_viz.color.r = 1.0
        self.msg_viz.color.g = 0.0
        self.msg_viz.color.b = 0.0
        
        self.hipCenter.position.x = 0.6
        self.hipCenter.position.y = 0.8
        
        print('lets dance')
        
        self.listener()
    def jointStatus_callback(self, data):
        global jointAngleOffset
        q1 = deg2rad(-float(data.relAngles[0]/4096.0*360.0+jointAngleOffset[0]))
        q2 = deg2rad(-float(data.relAngles[1]/4096.0*360.0+jointAngleOffset[1]))
        q3 = deg2rad(float(data.relAngles[2]/4096.0*360.0+jointAngleOffset[2]))
        q4 = deg2rad(float(data.relAngles[3]/4096.0*360.0+jointAngleOffset[3]))
        print(jointAngleOffset[0],jointAngleOffset[1],jointAngleOffset[2],jointAngleOffset[3])
        print(q1,q2,q3,q4)
        values = {lower_leg_length: 0.48, upper_leg_length: 0.54, hip_length: 0.175, theta0: 0, theta1: q1 , theta2: q2, theta3: q3, theta4: q4}
        Jpinv = J.subs(values).pinv()
        x_mes = Matrix([self.hipCenter.position.x, self.hipCenter.position.y])
        dq=Jpinv*( self.hip_desired + kp*(self.hip_desired - x_mes ))
        
        joint_msg = JointCommand()
        joint_msg.link_name.append('hip_1')
        joint_msg.link_name.append('knee_1')
        joint_msg.link_name.append('hip_2')
        joint_msg.link_name.append('knee_2')
        joint_msg.angle.append(q1+dq[0])
        joint_msg.angle.append(q2+dq[1])
        joint_msg.angle.append(q3+dq[2])
        joint_msg.angle.append(q4+dq[3])
        self.jointCommand_pub.publish(joint_msg)
        
        self.msg_viz.header.stamp = rospy.Time.now()
        ankle_left_pos = ankle_left_pos_equation.subs(values).to_matrix(inertial_frame)
        knee_left_pos = knee_left_pos_equation.subs(values).to_matrix(inertial_frame)
        hip_left_pos = hip_left_pos_equation.subs(values).to_matrix(inertial_frame)
        hip_right_pos = hip_right_pos_equation.subs(values).to_matrix(inertial_frame)
        knee_right_pos = knee_right_pos_equation.subs(values).to_matrix(inertial_frame)
        ankle_right_pos = ankle_right_pos_equation.subs(values).to_matrix(inertial_frame)

        self.msg_viz.id = 0
        del self.msg_viz.points[:]
        self.msg_viz.points.append(geometry_msgsPoint(ankle_left_pos[0],ankle_left_pos[1],ankle_left_pos[2]))
        self.msg_viz.points.append(geometry_msgsPoint(knee_left_pos[0],knee_left_pos[1],knee_left_pos[2]))
        self.visualization_pub.publish(self.msg_viz)

        self.msg_viz.id = 1
        del self.msg_viz.points[:]
        self.msg_viz.points.append(geometry_msgsPoint(knee_left_pos[0],knee_left_pos[1],knee_left_pos[2]))
        self.msg_viz.points.append(geometry_msgsPoint(hip_left_pos[0],hip_left_pos[1],hip_left_pos[2]))
        self.visualization_pub.publish(self.msg_viz)

        self.msg_viz.id = 2
        del self.msg_viz.points[:]
        self.msg_viz.points.append(geometry_msgsPoint(hip_left_pos[0],hip_left_pos[1],hip_left_pos[2]))
        self.msg_viz.points.append(geometry_msgsPoint(hip_right_pos[0],hip_right_pos[1],hip_right_pos[2]))
        self.visualization_pub.publish(self.msg_viz)

        self.msg_viz.id = 3
        del self.msg_viz.points[:]
        self.msg_viz.points.append(geometry_msgsPoint(hip_right_pos[0],hip_right_pos[1],hip_right_pos[2]))
        self.msg_viz.points.append(geometry_msgsPoint(knee_right_pos[0],knee_right_pos[1],knee_right_pos[2]))
        self.visualization_pub.publish(self.msg_viz)

        self.msg_viz.id = 4
        del self.msg_viz.points[:]
        self.msg_viz.points.append(geometry_msgsPoint(knee_right_pos[0],knee_right_pos[1],knee_right_pos[2]))
        self.msg_viz.points.append(geometry_msgsPoint(ankle_right_pos[0],ankle_right_pos[1],ankle_right_pos[2]))
        self.visualization_pub.publish(self.msg_viz)
        
    def hipCenter_callback(self, data):
        self.hipCenter = data
    def jointAngleOffset_callback(self,data):
        global jointAngleOffset
        jointAngleOffset = Matrix([data.angle[0],data.angle[1],data.angle[2],data.angle[3]])
        print (data.angle[0], data.angle[1], data.angle[2], data.angle[3])
    def hipCenterSetpoint_callback(self, data):
        self.hip_desired = Matrix([data.position.x, data.position.y])
        print (data.position.x, data.position.y)
    def listener(self):
        rospy.Subscriber("roboy/middleware/HipCenter", Pose, self.hipCenter_callback, queue_size=1)
        rospy.Subscriber("roboy/middleware/JointStatus", JointStatus, self.jointStatus_callback, queue_size=1)
        rospy.Subscriber("roboy/middleware/JointAngleOffset", JointAngle, self.jointAngleOffset_callback, queue_size=1)
        rospy.Subscriber("roboy/middleware/HipCenterSetpoint", Pose, self.hipCenterSetpoint_callback, queue_size=1)
        rospy.spin()

In [ ]:
danceController = PaBiRoboy_DanceController()

lets dance
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.18930153172
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.18930153172
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.18930153172
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.1908355125
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.18930153172
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.18930153172
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.18930153172
0 0 0 0
-2.73355376401 -2.0509323134 0.493941813699 4.18623357014
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.18930153172
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.18930153172
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.1908355125
0 0 0 0
-2.73355376401 -2.05246629419 0.492407832911 4.18930153172
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.19236949329
0 0 0 0
-2.73355376401 -2.05246629419 0.492407832911 4.1908355125
0 0 0 0
-2.73355376401 -2.0509323134 0.492407832911 4.18930153172


-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.3575650078
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.3575650078
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.3575650078
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.3258

-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32427709207 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32427709207 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28717939124 1.3575650078
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32

-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.3

-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.3575650078
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32427709207 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28717939124 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32

...

-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32427709207 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.3575650078
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32

-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.3575650078
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28717939124 1.36216695016
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32

-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.3575650078
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32427709207 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32

-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35603102701
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36216695016
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.3

-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36216695016
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36216695016
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35603102701
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.36063296937
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.32581107285 -1.28871337203 1.35909898858
-83.2834472656250 -193.473144531250 -102.050727844238 -162.158660888672
-1.27998339682 1.3

## Kinematical Differential Equations

In [ ]:
omega0, omega1, omega2, omega3, omega4 = dynamicsymbols('omega0, omega1, omega2, omega3, omega4')
kinematical_differential_equations = [omega0 - theta0.diff(),
                                      omega1 - theta1.diff(),
                                      omega2 - theta2.diff(),
                                      omega3 - theta3.diff(),
                                      omega4 - theta4.diff(),]

kinematical_differential_equations

## Angular Velocities

In [ ]:
hip_frame.set_ang_vel(inertial_frame,         omega0 * inertial_frame.z)
hip_frame.ang_vel_in(inertial_frame)

In [ ]:
upper_leg_left_frame.set_ang_vel(hip_frame,   omega1     * inertial_frame.z)
upper_leg_left_frame.ang_vel_in(inertial_frame)

In [ ]:
lower_leg_left_frame.set_ang_vel(upper_leg_left_frame,   omega2     * inertial_frame.z)
lower_leg_left_frame.ang_vel_in(inertial_frame)

In [ ]:
upper_leg_right_frame.set_ang_vel(hip_frame,             omega3     * inertial_frame.z)
upper_leg_right_frame.ang_vel_in(inertial_frame)

In [ ]:
lower_leg_right_frame.set_ang_vel(upper_leg_right_frame, omega4     * inertial_frame.z)
lower_leg_right_frame.ang_vel_in(inertial_frame)

# Linear Velocities

Finally, the linear velocities of the mass centers are needed. Starting at the hip_center which has a velocity of 0.

In [ ]:
hip_center.set_vel(inertial_frame, 0)

Working our way up the chain we can make use of the fact that the joint points are located on two rigid bodies. Any fixed point in a reference frame can be computed if the linear velocity of another point on that frame is known and the frame's angular velocity is known.

$$^I\mathbf{v}^{P_2} = ^I\mathbf{v}^{P_1} + ^I\omega^A \times \mathbf{r}^{\frac{P_2}{P_1}}$$

The `Point.v2pt_theory()` method makes it easy to do this calculation.

In [ ]:
hip_left.v2pt_theory(hip_center, inertial_frame, hip_frame)
hip_left.vel(inertial_frame)

In [ ]:
hip_right.v2pt_theory(hip_center, inertial_frame, hip_frame)
hip_right.vel(inertial_frame)

In [ ]:
hip_mass_center.v2pt_theory(hip_center, inertial_frame, hip_frame)
hip_mass_center.vel(inertial_frame)

In [ ]:
knee_left.v2pt_theory(hip_left, inertial_frame, upper_leg_left_frame)
knee_left.vel(inertial_frame)

In [ ]:
knee_right.v2pt_theory(hip_right, inertial_frame, upper_leg_right_frame)
knee_right.vel(inertial_frame)

In [ ]:
ankle_left.v2pt_theory(knee_left, inertial_frame, lower_leg_left_frame)
ankle_left.vel(inertial_frame)

In [ ]:
ankle_right.v2pt_theory(knee_right, inertial_frame, lower_leg_right_frame)
ankle_right.vel(inertial_frame)

In [ ]:
lower_leg_left_mass_center.v2pt_theory(knee_left, inertial_frame, lower_leg_left_frame)
lower_leg_left_mass_center.vel(inertial_frame)

In [ ]:
lower_leg_right_mass_center.v2pt_theory(knee_right, inertial_frame, lower_leg_right_frame)
lower_leg_right_mass_center.vel(inertial_frame)

In [ ]:
upper_leg_left_mass_center.v2pt_theory(knee_left, inertial_frame, upper_leg_left_frame)
upper_leg_left_mass_center.vel(inertial_frame)

In [ ]:
upper_leg_right_mass_center.v2pt_theory(knee_right, inertial_frame, upper_leg_right_frame)
upper_leg_right_mass_center.vel(inertial_frame)

# Masses, Inertia, Rigid Bodies

In [ ]:
lower_leg_mass, upper_leg_mass, hip_mass = symbols('m_L, m_U, m_H')

In [ ]:
lower_leg_inertia, upper_leg_inertia, hip_inertia = symbols('I_Lz, I_Uz, I_Hz')

In [ ]:
lower_leg_left_inertia_dyadic = inertia(lower_leg_left_frame, 0, 0, lower_leg_inertia)
lower_leg_left_central_inertia = (lower_leg_left_inertia_dyadic, lower_leg_left_mass_center)
lower_leg_left_inertia_dyadic.to_matrix(lower_leg_left_frame)

In [ ]:
upper_leg_left_inertia_dyadic = inertia(upper_leg_left_frame, 0, 0, upper_leg_inertia)
upper_leg_left_central_inertia = (upper_leg_left_inertia_dyadic, upper_leg_left_mass_center)
upper_leg_left_inertia_dyadic.to_matrix(upper_leg_left_frame)

In [ ]:
hip_inertia_dyadic = inertia(hip_frame, 0, 0, hip_inertia)
hip_central_inertia = (hip_inertia_dyadic, hip_mass_center)
hip_inertia_dyadic.to_matrix(hip_frame)

In [ ]:
upper_leg_right_inertia_dyadic = inertia(upper_leg_right_frame, 0, 0, upper_leg_inertia)
upper_leg_right_central_inertia = (upper_leg_right_inertia_dyadic, upper_leg_right_mass_center)
upper_leg_right_inertia_dyadic.to_matrix(upper_leg_right_frame)

In [ ]:
lower_leg_right_inertia_dyadic = inertia(lower_leg_right_frame, 0, 0, lower_leg_inertia)
lower_leg_right_central_inertia = (lower_leg_right_inertia_dyadic, lower_leg_right_mass_center)
lower_leg_right_inertia_dyadic.to_matrix(lower_leg_right_frame)

In [ ]:
lower_leg_left = RigidBody('Lower Leg Left', lower_leg_left_mass_center, lower_leg_left_frame,
                      lower_leg_mass, lower_leg_left_central_inertia)

In [ ]:
upper_leg_left = RigidBody('Upper Leg Left', upper_leg_left_mass_center, upper_leg_left_frame,
                      upper_leg_mass, upper_leg_left_central_inertia)

In [ ]:
hip = RigidBody('Hip', hip_mass_center, hip_frame, hip_mass, hip_central_inertia)

In [ ]:
upper_leg_right = RigidBody('Upper Leg Right', upper_leg_right_mass_center, upper_leg_right_frame,
                      upper_leg_mass, upper_leg_right_central_inertia)

In [ ]:
lower_leg_right = RigidBody('Lower Leg Right', lower_leg_right_mass_center, lower_leg_right_frame,
                      lower_leg_mass, lower_leg_right_central_inertia)

# Forces and Torques

In [ ]:
g = symbols('g')

In [ ]:
lower_leg_left_grav_force = (lower_leg_left_mass_center, -lower_leg_mass * g * inertial_frame.y)
upper_leg_left_grav_force = (upper_leg_left_mass_center, -upper_leg_mass * g * inertial_frame.y)
hip_grav_force = (hip_mass_center, -hip_mass * g * inertial_frame.y)
upper_leg_right_grav_force = (upper_leg_right_mass_center, -upper_leg_mass * g * inertial_frame.y)
lower_leg_right_grav_force = (lower_leg_right_mass_center, -lower_leg_mass * g * inertial_frame.y)

In [ ]:
ankle_torque0, knee_torque0, hip_torque0, hip_torque1, knee_torque1, ankle_torque1 = dynamicsymbols('T_a0, T_k0, T_h0, T_h1, T_k1, T_a1')

In [ ]:
lower_leg_left_torque_vector = ankle_torque0 * inertial_frame.z - knee_torque0 * inertial_frame.z
upper_leg_left_torque_vector = knee_torque0 * inertial_frame.z - hip_torque0 * inertial_frame.z
hip_left_torque_vector = hip_torque0 * inertial_frame.z - hip_torque1 * inertial_frame.z
hip_right_torque_vector = hip_torque1 * inertial_frame.z - knee_torque1 * inertial_frame.z
upper_leg_right_torque_vector = knee_torque1 * inertial_frame.z - ankle_torque1 * inertial_frame.z
lower_leg_right_torque_vector = ankle_torque1 * inertial_frame.z

In [ ]:
lower_leg_left_torque = (lower_leg_left_frame, lower_leg_left_torque_vector)
upper_leg_left_torque = (upper_leg_left_frame, upper_leg_left_torque_vector)
hip_left_torque = (hip_frame, hip_left_torque_vector)
hip_right_torque = (hip_frame, hip_right_torque_vector)
upper_leg_right_torque = (upper_leg_right_frame, upper_leg_right_torque_vector)
lower_leg_right_torque = (lower_leg_right_frame, lower_leg_right_torque_vector)

# Equations of Motion

In [ ]:
coordinates = [theta0, theta1, theta2, theta3, theta4]
coordinates

In [ ]:
speeds = [omega0, omega1, omega2, omega3, omega4]
speeds

In [ ]:
kinematical_differential_equations

In [ ]:
kane = KanesMethod(inertial_frame, coordinates, speeds, kinematical_differential_equations)

In [ ]:
loads = [lower_leg_left_grav_force,
         upper_leg_left_grav_force,
         hip_grav_force, 
         upper_leg_right_grav_force,
         lower_leg_right_grav_force,
         lower_leg_left_torque,
         upper_leg_left_torque,
         hip_left_torque,
         hip_right_torque,
         upper_leg_right_torque,
         lower_leg_right_torque]
loads

In [ ]:
bodies = [lower_leg_left, upper_leg_left, hip, upper_leg_right, lower_leg_right]
bodies

In [ ]:
fr, frstar = kane.kanes_equations(loads, bodies)

In [ ]:
trigsimp(fr + frstar)

Keep in mind that out utlimate goal is to have the equations of motion in first order form:

$$ \dot{\mathbf{x}} = \mathbf{g}(\mathbf{x}, t) $$

The equations of motion are linear in terms of the derivatives of the generalized speeds and the `KanesMethod` class automatically puts the equations in a more useful form for the next step of numerical simulation:

$$ \mathbf{M}(\mathbf{x}, t)\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x}, t) $$

Note that

$$ \mathbf{g} = \mathbf{M}^{-1}(\mathbf{x}, t) \mathbf{f}(\mathbf{x}, t) $$

and that $\mathbf{g}$ can be computed analytically but for non-toy problems, it is best to do this numerically. So we will simply generate the $\mathbf{M}$ and $\mathbf{f}$ matrices for later use.

The mass matrix, $\mathbf{M}$, can be accessed with the `mass_matrix` method (use `mass_matrix_full` to include the kinematical differential equations too. We can use `trigsimp` again to make this relatively compact: 

In [ ]:
mass_matrix = trigsimp(kane.mass_matrix_full)
mass_matrix

The right hand side, $\mathbf{f}$, is a vector function of all the non-inertial forces (gyroscopic, external, coriolis, etc):

In [ ]:
forcing_vector = trigsimp(kane.forcing_full)
forcing_vector

# Simulation

In [ ]:
constants = [lower_leg_length,
             upper_leg_length,
             hip_length,
             
             lower_leg_left_com_length,
             upper_leg_left_com_length,
             hip_com_length,
             upper_leg_right_com_length,
             lower_leg_right_com_length,
             
             lower_leg_mass,
             upper_leg_mass,
             hip_mass,
             
             lower_leg_inertia,
             upper_leg_inertia,
             hip_inertia,
             
             hip_offset_x,
             hip_offset_y,
             
             g]
constants

In [ ]:
numerical_constants = array([0.42,  # lower_leg_length [m]
                             0.54, # upper_leg_length [m]
                             0.2, # hip_length
                             
                             0.21,  # lower_leg_com_length [m]
                             0.27,  # upper_leg_com_length
                             0.10,  # hip_com_length
                             0.27,  # upper_leg_com_length
                             0.21,  # lower_leg_com_length [m]
                             
                             1.0,  # lower_leg_mass [kg]
                             1.5,  # upper_leg_mass [kg]
                             2.0,  # hip_mass [kg]
                             
                             0.1,  # lower_leg_inertia [kg*m^2]                             
                             0.2,  # upper_leg_inertia [kg*m^2]
                             0.1,  # hip_inertia [kg*m^2]
                             
                             0,
                             0.6,
                             
                             9.81],  # acceleration due to gravity [m/s^2]
                            ) 
numerical_constants

In [ ]:
coordinates = [theta0, theta1, theta2, theta3, theta4]
coordinates

In [ ]:
speeds = [omega0, omega1, omega2, omega3, omega4]
speeds

In [ ]:
specified = [ankle_torque0, knee_torque0, hip_torque0, hip_torque1, knee_torque1, ankle_torque1]

In [ ]:
right_hand_side = generate_ode_function(forcing_vector, coordinates,
                                        speeds, constants,
                                        mass_matrix=mass_matrix,
                                        specifieds=specified)

In [ ]:
x0 = zeros(10)
x0

In [ ]:
x0[0] = 0
x0[1] = deg2rad(90)
x0[2] = deg2rad(90)
x0[3] = deg2rad(-90)
x0[4] = deg2rad(-90)
x0

In [ ]:
numerical_specified = zeros(6)

args = {'constants': numerical_constants,
        'specified': numerical_specified}
frames_per_sec = 60
final_time = 5.0

t = linspace(0.0, final_time, final_time * frames_per_sec)

In [ ]:
right_hand_side(x0, 0.0, numerical_specified, numerical_constants)

In [ ]:
y = odeint(right_hand_side, x0, t, args=(numerical_specified, numerical_constants))
y.shape

In [ ]:
plot(t, rad2deg(y[:, :5]))
xlabel('Time [s]')
ylabel('Angle [deg]')
legend(["${}$".format(vlatex(c)) for c in coordinates])

In [ ]:
plot(t, rad2deg(y[:, 5:]))
xlabel('Time [s]')
ylabel('Angular Rate [deg/s]')
legend(["${}$".format(vlatex(s)) for s in speeds])

# Visualization

In [ ]:
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point
visualization_pub = rospy.Publisher('/visualization_marker', Marker, queue_size=1)
jointCommand_pub = rospy.Publisher('/roboy/middleware/JointCommand', JointCommand, queue_size=1)
msg = Marker()
msg.header.frame_id = "world"
msg.header.stamp = rospy.Time.now()
msg.type = Marker.ARROW
msg.scale.x = 0.01
msg.scale.y = 0.05
msg.scale.z = 0.05
msg.color.a = 0.3
msg.color.r = 1.0
msg.color.g = 0.0
msg.color.b = 0.0

In [ ]:
for i in range(0,int(final_time * frames_per_sec)):#
    values = {hip_offset_x: 0, hip_offset_y: 0.6, lower_leg_length: 0.4, upper_leg_length: 0.54, hip_length: 0.2, theta0: y[i, 0], theta1: y[i, 1], theta2: y[i, 2], theta3: y[i, 3], theta4: y[i, 4]}
    
    joint_msg = JointCommand()
    joint_msg.link_name.append('hip_1')
    joint_msg.link_name.append('knee_1')
    joint_msg.link_name.append('hip_2')
    joint_msg.link_name.append('knee_2')
    joint_msg.angle.append(y[i, 1])
    joint_msg.angle.append(y[i, 2])
    joint_msg.angle.append(y[i, 3])
    joint_msg.angle.append(y[i, 4])
    jointCommand_pub.publish(joint_msg)
    
    ankle_left_pos = ankle_left_pos_equation.subs(values).to_matrix(inertial_frame)
    knee_left_pos = knee_left_pos_equation.subs(values).to_matrix(inertial_frame)
    hip_left_pos = hip_left_pos_equation.subs(values).to_matrix(inertial_frame)
    hip_right_pos = hip_right_pos_equation.subs(values).to_matrix(inertial_frame)
    knee_right_pos = knee_right_pos_equation.subs(values).to_matrix(inertial_frame)
    ankle_right_pos = ankle_right_pos_equation.subs(values).to_matrix(inertial_frame)
    
    msg.id = 0
    del msg.points[:]
    msg.points.append(Point(ankle_left_pos[0],ankle_left_pos[1],ankle_left_pos[2]))
    msg.points.append(Point(knee_left_pos[0],knee_left_pos[1],knee_left_pos[2]))
    visualization_pub.publish(msg)
    
    msg.id = 1
    del msg.points[:]
    msg.points.append(Point(knee_left_pos[0],knee_left_pos[1],knee_left_pos[2]))
    msg.points.append(Point(hip_left_pos[0],hip_left_pos[1],hip_left_pos[2]))
    visualization_pub.publish(msg)
    
    msg.id = 2
    del msg.points[:]
    msg.points.append(Point(hip_left_pos[0],hip_left_pos[1],hip_left_pos[2]))
    msg.points.append(Point(hip_right_pos[0],hip_right_pos[1],hip_right_pos[2]))
    visualization_pub.publish(msg)
    
    msg.id = 3
    del msg.points[:]
    msg.points.append(Point(knee_right_pos[0],knee_right_pos[1],knee_right_pos[2]))
    msg.points.append(Point(hip_right_pos[0],hip_right_pos[1],hip_right_pos[2]))
    visualization_pub.publish(msg)
    
    msg.id = 4
    del msg.points[:]
    msg.points.append(Point(ankle_right_pos[0],ankle_right_pos[1],ankle_right_pos[2]))
    msg.points.append(Point(knee_right_pos[0],knee_right_pos[1],knee_right_pos[2]))
    visualization_pub.publish(msg)
    
